In [ ]:
# want to see the images inline
# %matplotlib inline

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import gamma
from scipy.optimize import curve_fit
from scipy import special

from configparser import ConfigParser, ExtendedInterpolation

import utils as utils

from tqdm import tqdm_notebook as tqdm

In [ ]:
# read config file 
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config.ini')

In [ ]:
PARAMS = config.get('main', 'PARAMS')

In [ ]:
FOLDER = config.get(PARAMS, 'FOLDER')
EXPERIMENT = config.get(PARAMS, 'EXPERIMENT')
DB_FILENAME = config.get(PARAMS, 'DB_FILENAME')
TYPE = config.get(PARAMS, 'TYPE')
COLOR = config.get(PARAMS, 'COLOR')
MAKE_IMAGES = config.getboolean('main', 'MAKE_IMAGES')

In [ ]:
# some const params for all graphs
num_bins = 100
# graph [xmin, xmax]
xmin = 0
xmax = 100
binwidth = (xmax - xmin)/num_bins

bins = np.arange(xmin, xmax + binwidth, binwidth)
print ('bins: ', bins.shape)

In [ ]:
# important indices
type_columns = ['c0_type', 'c1_type', 'c2_type']
stain_prefix = np.array([['C0-', 'C1-', 'C2-', 'C3-', 'C4-']])
filename_column = 'cropped_image_file'

In [ ]:
# read the db and parse images that we want to process
df_path = os.path.join(FOLDER, 'smFISH-database', DB_FILENAME)
df = pd.read_csv(df_path, 
                 sep=',', 
                )

In [ ]:
# TODO: maybe it is necessary to fill in other values here, too
# fix missing entries in the colmns that we are planning to use 
df['cropped_image_file'].fillna('', inplace=True)
df['c0_type'].fillna('', inplace=True)
df['c1_type'].fillna('', inplace=True)
df['c2_type'].fillna('', inplace=True)

In [ ]:
dff = df[df['cropped_image_file'].apply(lambda x: x.startswith(EXPERIMENT))]

In [ ]:
# seems to be working
row, col = np.where(dff[type_columns].applymap(lambda x: x == TYPE))
n_samples = dff.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = dff[filename_column].values[row]
full_filenames = ["{}{}".format(a_, b_[:-4]) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
dataset = []
pbar = tqdm(total=len(full_filenames))
for ff in full_filenames: 
    tmp = os.path.join(FOLDER, "csv-2", ff + ".csv")
    dataset.append(tmp)
    pbar.update(1)
pbar.close()

In [ ]:
# center_path = os.path.join(FOLDER, "centers", "all-centers.csv")

In [ ]:
# d = '/Volumes/MILKYKLIM/2019-05-06-exp/SEA-12/csv-2/C2-SEA-12_277_cropped_4.csv'
# I, z = utils.load_i_z(d, skiprows_=0)
# print(np.round(z))

In [ ]:
print(f'Processing: {len(dataset)} files') 

# actual plotting 
pbar = tqdm(total=len(dataset))
for d in dataset:    
    pbar.update(1) 
    if(not os.path.exists(d)):
        continue
        
    try:
        # create the canvas
        fig = plt.figure(figsize=(8,5))
        title = utils.create_title(d, name_id=-1)
        fig.suptitle(title + " / " + TYPE)
        
        # load the data and scale it accordingly
        I, z = utils.load_i_z(d, skiprows_=0)

        # calculate the params for gauss fit
        binned_values, real_bins = np.histogram(z, bins)

        plt.hist(z, 
                 bins=bins, 
                 color=COLOR, 
                 label=TYPE, 
                 density=False,
                )
        plt.grid(True, lw = 2, ls = '--', c = '.85')
        
        # reasonable adjustments to make the data look nicer
        plt.xlabel('z')
        plt.ylabel('# spots')

        x_limits = [xmin, xmax]
        ymax = 45 # np.max(np.histogram(z, bins)[0])
        y_limits = [0, ymax]

        plt.xlim(x_limits)
        plt.ylim(y_limits)
        plt.xticks(np.arange(xmin, xmax + binwidth, binwidth*10))

        folder_path = os.path.join(FOLDER, "z-counts", TYPE)
        os.makedirs(folder_path, exist_ok=True)
            
        plt.savefig(os.path.join(folder_path, title + ".pdf"))
        # plt.show()
        # break
        plt.close()
    except(RuntimeError, TypeError, ValueError):
        print("There was an exception but we\'ll fix it for you")
pbar.close()
    

In [ ]:
TYPE

In [ ]:
PARAMS